<a href="https://colab.research.google.com/github/bntumb/Bank-Marketing-Data-Set/blob/main/Logistic_Regression_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>
Understanding Euler's number and natural logarithm

In [ ]:
#what is e? it is the base rate of growth shared by all processes that continue to grow
import math 
math.e

2.718281828459045


For example, if I have 5 GBP, and it grows 100% in 1 year (compounding continuously), I end up with 5*e^1 GBP:

In [ ]:
#100% growth for the 1st year
5*np.exp(1)

NameError: ignored

In [ ]:
#100% growth for the 2nd year
5*np.exp(2)

In [ ]:
#visualise the data
# Plot this as a curve
plt.plot([np.exp(x) for x in range(5)])

The natural logarithm gives you the time needed to reach a certain level of growth. E.g. if I want to growth by a factor of e then it will take 1 unit of time assuming 100% growth rate

In [ ]:
#grow 1 unit to 2.718
np.log(2.718)

In [ ]:
#grow by 2 units
np.log(7.389)

Exponential and natural log are inverse of each other

<b> 
Probability, Odds and Log Odds

Before we can understand how logistic regression predicts its outputs, we need to understand how probability, odds ratios and log odds are related. This is because logistic regression is an algorithm that predicts log odds and our ability to understand that helps us understand logistic regression. We can think of numeric odds as ratios e.g. 5/1 odds means 5 to 1 for every 6 plays or 2/3 means 2 to 3 for every 5 plays.

Examples:

Dice roll of 1: probability = 1/6, odds = 1/5 = 0.2

probability = odds / (1+ odds)

Let's look at the difference between probabilities and odds in the below code

Calculate the probability, odds and log odds in an ordinary deck of cards:
(Hint : You may want to use the log function from NumPy)
- 2 of spade
- Hearts
- Card with Queen on it

In [ ]:
#A 2 of spades - there are 52 cards in a deck, how many 2 of spades are there? 
#probability
1/52

In [ ]:
#odds
1/51

In [ ]:
#log odds
np.log(0.02)

In [ ]:
# all cards are divided into 4 suits, in each suit there are 13 cards which means there are 13 heart cards
13/52

In [ ]:
#odds
13/39

In [ ]:
#log odds
np.log(0.33)

In [ ]:
# if there are 4 suits and each suit has a queen then there are 4 queens
4/52

In [ ]:
4/48

In [ ]:
np.log(0.083)

Linear models are based on ordinary least squares algorithms - plotting a function as close to modelling most of the observed data points in the problem domain 

Logistic regression: log odds of a categorical response being true is modeled as a linear combination of the features aka a logit function.

In simple terms:
- Logistic regression outputs the probabilities of a specific class.
- Those probabilities can be converted into class predictions.

The logistic function has some nice properties:
- Takes on an "s" shape
- Output is bounded by 0 and 1

Use Pandas to load in the direct marketing data from the UCI machine learning website.
The columns are different attributes which helps determine whether the client will subscribe to a term deposit (1 = yes, 0 = no)

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [ ]:
data = pd.read_csv('bank.csv')

FileNotFoundError: ignored

In [ ]:
data.head()

In [ ]:
#start exploring the data
data['y'].value_counts()

In [ ]:
sns.countplot(x = 'y', data = data, palette = 'hls')
plt.show()
plt.savefig('count_plot')

In [ ]:
data['y'] = data['y'].map({'yes': 1, 'no': 0})

In [ ]:
#start exploring the data
data.head()

In [ ]:
count_no_sub = len(data[data['y']==0])
count_sub = len(data[data['y']==1])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("percentage of no subscription is", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("percentage of subscription", pct_of_sub*100)

Our classes are imbalanced, and the ratio of no-subscription to subscription instances is 89:11. Before we go ahead to balance the classes, let’s do some more exploration.

In [ ]:
data.groupby('y').mean()

what are your observations?

- look at age
- look at when the customers were contacted

https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

In [ ]:
data.groupby('job').mean()

In [ ]:
data.groupby('marital').mean()

In [ ]:
data.groupby('education').mean()

Visualisation

In [ ]:
%matplotlib inline
pd.crosstab(data.job,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Job Title')
plt.xlabel('Job')
plt.ylabel('Frequency of Purchase')

The frequency of purchase of the deposit depends a great deal on the job title. Thus, the job title can be a good predictor of the outcome variable.

In [ ]:
table=pd.crosstab(data.marital,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Marital Status vs Purchase')
plt.xlabel('Marital Status')
plt.ylabel('Proportion of Customers')


The marital status does not seem a strong predictor for the outcome variable.

In [ ]:
table=pd.crosstab(data.education,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Education vs Purchase')
plt.xlabel('Education')
plt.ylabel('Proportion of Customers')
plt.savefig('edu_vs_pur_stack')

In [ ]:
pd.crosstab(data.month,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Month')
plt.xlabel('Month')
plt.ylabel('Frequency of Purchase')

In [ ]:
data.age.hist()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')

In [ ]:
pd.crosstab(data.poutcome,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Poutcome')
plt.xlabel('Poutcome')
plt.ylabel('Frequency of Purchase')

Create dummy variables
That is variables with only two values, zero and one.

In [ ]:
cat_vars=['job','marital','education','default','housing','loan','contact','month','poutcome']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1
cat_vars=['job','marital','education','default','housing','loan','contact','month','poutcome']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]


Confirm that this is the same value we would get when using the built-in .predict() method of the LinearRegression object.

In [ ]:
data_final=data[to_keep]
data_final.columns.values

In [ ]:
data_final.head()

Over-sampling using SMOTE
With our training data created, we will up-sample the no-subscription using the SMOTE algorithm(Synthetic Minority Oversampling Technique). 
At a high level, SMOTE:
1. Works by creating synthetic samples from the minor class (no-subscription) instead of creating copies.
2. Randomly choosing one of the k-nearest-neighbors and using it to create a similar, but randomly tweaked, new observations.
We are going to implement SMOTE in Python.

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)
import imblearn; print("ImbLearn", imblearn.__version__)

In [ ]:
X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']
from imblearn.over_sampling import SMOTE
os = SMOTE()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )  
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

In [ ]:
os.fit_resample

In [ ]:
# you had 
os_data_X = pd.DataFrame(data=X_train,columns=columns )  ## have you changed these lines from the original? data=X_train  
os_data_y= pd.DataFrame(data=y_train,columns=['y'])
# but it should be 
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )  
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

Can you see what we have done here? Using SMOTE we have actually synthetically created a perfectly balanced set if data. We oversampled on the training data so that no information in the test data is used to create synthetic observations. 


Next we will look at recursive feature elimination - this is based on the idea to repeatedly construct a model and choose either the best or worst performing feature, setting the feature aside and then repeating the process with the rest of the features. This process is applied until all features in the dataset are exhausted. The goal of RFE is to select features by recursively considering smaller and smaller sets of features.

In [ ]:
pip install --upgrade pip

In [ ]:
data_final_vars=data_final.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
print(X)

In [ ]:
np.array(X)[rfe.support_]

Certain features have been selected as part of the RFE

In [ ]:
cols=['job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married',
       'marital_single', 'education_primary', 'education_secondary',
       'education_tertiary', 'education_unknown', 'housing_no',
       'housing_yes']

X=os_data_X[cols]
y=os_data_y['y']

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

<b>
Logistic Regression Model Fitting

The values from a linear regression cannot be interpreted as probabilites of class membership since their values can be greater than 1 or less than 0. Logistic regression ensures that the outputs are predicted as the likelihood it will be classed in that category

We start by importing a logistic regression class from the sklearn package and fit the same regression model with what we did above

In [ ]:
# Fit a logistic regression model and store the class predictions.
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

Predicting the test set results and calculating the accuracy

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

When we evaluate the performance of a logistic regression (or any classifier model), the standard metric to use is accuracy: How many class labels did we guess correctly? However, accuracy is only one of several metrics we could use when evaluating a classification model.

$$Accuracy = \frac{total~predicted~correct}{total~predicted}$$

Accuracy alone doesn’t always give us a full picture.

If we know a model is 75% accurate, it doesn’t provide any insight into why the 25% were wrong.

Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

The result is telling us that we have 8053+315 correct predictions and 1230+7174 incorrect predictions.

Compute precision, recall, F-measure and support


To quote from Scikit Learn:


The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the 

classifier to not label a sample as positive if it is negative.


The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the 

classifier to find all the positive samples.

The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.

The F-beta score weights the recall more than the precision by a factor of beta. beta = 1.0 means recall and precision are equally important.


The support is the number of occurrences of each class in y_test.

more info on confusion matrix https://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Interpretation: Of the entire test set, 91% of the promoted term deposit were the term deposit that the customers liked. Of the entire test set, 91% of the customer’s preferred term deposits that were promoted.

ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

The receiver operating characteristic (ROC) curve is another common tool used with binary classifiers. The dotted line represents the ROC curve of a purely random classifier; a good classifier stays as far away from that line as possible (toward the top-left corner).

Comparing Logistic Regression to Other Models
Advantages of logistic regression:

Highly interpretable (if you remember how).
Model training and prediction are fast.
No tuning is required (excluding regularisation).
Features don't need scaling.
Can perform well with a small number of observations.
Outputs well-calibrated predicted probabilities.
Disadvantages of logistic regression:

Presumes a linear relationship between the features and the log odds of the response.
Performance is (generally) not competitive with the best supervised learning methods.

<b> 
Understanding size of samples

rule of thumb - logistic regression typically requires a large sample size.  

A general guideline is that you need at minimum of 10 cases with the least frequent outcome for each independent variable in your model. 

For example, if you have 5 independent variables and the expected probability of your least frequent outcome is .10, then you would need a minimum sample size of 500 (10*5 / .10)

<b>
statistical testing

Logistic regression is one of the few machine learning models where we can obtain comprehensive statistics. By performing hypothesis testing, we can understand whether we have sufficient data to make strong conclusions about individual coefficients and the model as a whole. A very popular Python library which gives you these statistics with just a few lines of code is statsmodels.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8a08b6fc-4059-46d3-b140-324ad35bca75' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>